In [5]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import datetime

In [6]:
stock_data = pd.read_csv('Stockreturn.csv')
return_data = pd.read_csv('OutstandingShares.csv')
mapping_data = pd.read_csv('Companyprofiles.csv')

In [7]:
columns = ['Ticker', 'time', 'Open', 'High', 'Low','Close']
stock_data = stock_data[columns]
stock_data.head(10)

,Ticker,time,Open,High,Low,Close
0,AALB.AS,20200401,20.92,26.48,20.53,25.70
1,AALB.AS,20191201,38.59,40.92,37.08,40.01
2,AALB.AS,20200201,39.57,43.11,32.96,33.75
3,AALB.AS,20200101,40.01,41.06,39.13,39.52
4,AALB.AS,20200601,25.00,30.89,24.75,29.19
5,AALB.AS,20200301,34.51,34.68,16.16,21.64
6,AALB.AS,20200501,24.74,25.60,21.15,24.65
7,AALB.AS,20191101,36.15,39.80,36.01,38.45
8,AALB.AS,20191001,36.40,37.96,34.05,36.06
9,ABN.AS,20200401,7.24,8.30,6.23,7.00


In [8]:
#Left merge tables on ticker
new_table = stock_data.merge(right = return_data, on='Ticker', how='left')
new_table.head(5)

#Left merge tables on ticker to add industry mappings
new_table = mapping_data.merge(right = new_table, on='Ticker', how='left')
new_table.head(5)
new_table.time = new_table.time.fillna(0).astype(int)

In [9]:
new_table = new_table[new_table['time'] !=0]

In [10]:
new_table.time = new_table.time.fillna(0).astype(int).apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d')  )

In [11]:
#Change all column names to lower case
new_table.columns= new_table.columns.str.lower()
new_table.head(5)

,company,full time employees,industry,sector(s),ticker,time,open,high,low,close,unnamed: 0,index,basic average shares
0,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2020-04-01,20.92,26.48,20.53,25.70,2.00,"2,019.00","110,580.00"
1,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2019-12-01,38.59,40.92,37.08,40.01,2.00,"2,019.00","110,580.00"
2,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2020-02-01,39.57,43.11,32.96,33.75,2.00,"2,019.00","110,580.00"
3,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2020-01-01,40.01,41.06,39.13,39.52,2.00,"2,019.00","110,580.00"
4,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2020-06-01,25.00,30.89,24.75,29.19,2.00,"2,019.00","110,580.00"


In [19]:
#Calculate market value
new_table['market value low'] = new_table['low'] * new_table['basic average shares']
new_table['market value high'] = new_table['high'] * new_table['basic average shares']
new_table['difference'] = new_table['market value high'] - new_table['market value low']
new_table.head(5)

,company,full time employees,industry,sector(s),ticker,time,open,high,low,close,unnamed: 0,index,basic average shares,market value,market value low,market value high,difference
0,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2020-04-01,20.92,26.48,20.53,25.70,2.00,"2,019.00","110,580.00","2,841,906.00","2,270,207.40","2,928,158.40","657,951.00"
1,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2019-12-01,38.59,40.92,37.08,40.01,2.00,"2,019.00","110,580.00","4,424,305.80","4,100,306.40","4,524,933.60","424,627.20"
2,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2020-02-01,39.57,43.11,32.96,33.75,2.00,"2,019.00","110,580.00","3,732,075.00","3,644,716.80","4,767,103.80","1,122,387.00"
3,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2020-01-01,40.01,41.06,39.13,39.52,2.00,"2,019.00","110,580.00","4,370,121.60","4,326,995.40","4,540,414.80","213,419.40"
4,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2020-06-01,25.00,30.89,24.75,29.19,2.00,"2,019.00","110,580.00","3,227,830.20","2,736,855.00","3,415,816.20","678,961.20"


In [21]:
df = new_table.groupby(['time'])['market value low','market value high', 'difference'].sum()
df

,market value low,market value high,difference
time,,,
2019-10-01,"1,234,301,108.30","1,354,831,722.23","120,530,613.93"
2019-11-01,"1,270,707,975.59","1,363,209,784.26","92,501,808.67"
2019-12-01,"1,265,098,959.77","1,370,786,551.14","105,687,591.36"
2020-01-01,"1,252,614,007.69","1,404,959,092.14","152,345,084.45"
2020-02-01,"1,120,767,484.08","1,377,524,679.38","256,757,195.30"
2020-03-01,"735,755,915.06","1,211,430,582.95","475,674,667.89"
2020-04-01,"907,701,392.22","1,104,870,704.68","197,169,312.46"
2020-05-01,"946,846,891.80","1,121,020,554.51","174,173,662.71"
2020-06-01,"1,031,724,433.11","1,207,374,872.95","175,650,439.84"


In [33]:
df2 = new_table.groupby(['sector(s)', 'time'])['market value low','market value high', 'difference',].sum()
df2

market value low  market value high  \
sector(s)              time                                              
Basic Materials        2019-10-01     58,547,307.06      65,119,071.59   
                       2019-11-01     61,169,522.68      69,657,046.20   
                       2019-12-01     64,601,611.30      71,149,636.70   
                       2020-01-01     62,200,455.08      70,337,216.52   
                       2020-02-01     56,935,327.35      71,410,144.64   
                       2020-03-01     37,632,232.23      62,181,893.19   
                       2020-04-01     45,913,305.27      58,161,360.58   
                       2020-05-01     50,001,380.59      59,433,024.65   
                       2020-06-01     54,654,726.92      62,712,806.27   
Communication Services 2019-10-01    140,857,830.80     156,845,410.21   
                       2019-11-01    141,431,630.13     152,519,734.92   
                       2019-12-01    140,869,918.48     158,158,481.70   
                       2020-01-01    150,650,036.64     168,814,828.75   
                       2020-02-01    141,236,975.80     171,476,046.47   
                       2020-03-01    102,532,395.54     152,870,899.10   
                       2020-04-01    127,660,596.60     152,289,411.10   
                       2020-05-01    139,970,346.79     168,076,989.77   
                       2020-06-01    149,264,690.57     171,578,961.79   
Consumer Cyclical      2019-10-01      9,867,922.62      10,366,271.27   
                       2019-11-01     10,027,972.88      10,647,013.90   
                       2019-12-01     10,291,131.93      10,678,554.05   
                       2020-01-01     10,425,749.83      10,858,914.17   
                       2020-02-01     10,088,051.09      10,875,679.61   
                       2020-03-01      5,625,836.69      10,398,490.22   
                       2020-04-01      7,211,088.88       8,731,321.70   
                       2020-05-01      7,874,974.14       9,199,001.61   
                       2020-06-01      8,397,127.69       9,472,111.45   
Consumer Defensive     2019-10-01    261,767,121.81     284,149,438.10   
                       2019-11-01    261,242,935.22     276,805,225.64   
                       2019-12-01    258,993,160.57     276,959,286.71   
...                                             ...                ...   
Industrials            2020-04-01     30,830,875.88      39,451,446.63   
                       2020-05-01     32,453,624.66      41,202,703.07   
                       2020-06-01     38,650,218.28      46,973,694.34   
Real Estate            2019-10-01     33,090,743.46      36,795,467.23   
                       2019-11-01     34,989,832.26      37,284,933.06   
                       2019-12-01     34,474,902.07      36,462,555.48   
                       2020-01-01     32,463,056.31      36,830,462.14   
                       2020-02-01     29,991,281.72      35,799,107.71   
                       2020-03-01     16,014,507.69      32,027,179.42   
                       2020-04-01     16,843,104.75      22,484,991.89   
                       2020-05-01     15,721,324.62      19,397,865.19   
                       2020-06-01     17,780,507.05      23,599,301.95   
Technology             2019-10-01    118,097,029.35     134,001,551.53   
                       2019-11-01    128,532,217.67     138,096,120.25   
                       2019-12-01    130,564,151.10     147,012,959.41   
                       2020-01-01    139,892,501.71     154,288,462.29   
                       2020-02-01    134,572,218.63     164,450,405.22   
                       2020-03-01    100,874,952.29     150,345,792.54   
                       2020-04-01    122,327,754.63     157,293,382.90   
                       2020-05-01    141,190,606.16     173,291,684.69   
                       2020-06-01    165,551,136.89     191,544,521.03   
Utilities              2019-10-01      

In [36]:
df3 = new_table.groupby(['sector(s)', 'time'])['difference',].sum()
df3

difference
sector(s)              time                    
Basic Materials        2019-10-01  6,571,764.53
                       2019-11-01  8,487,523.52
                       2019-12-01  6,548,025.40
                       2020-01-01  8,136,761.44
                       2020-02-01 14,474,817.30
                       2020-03-01 24,549,660.96
                       2020-04-01 12,248,055.31
                       2020-05-01  9,431,644.05
                       2020-06-01  8,058,079.36
Communication Services 2019-10-01 15,987,579.41
                       2019-11-01 11,088,104.79
                       2019-12-01 17,288,563.22
                       2020-01-01 18,164,792.11
                       2020-02-01 30,239,070.66
                       2020-03-01 50,338,503.56
                       2020-04-01 24,628,814.50
                       2020-05-01 28,106,642.98
                       2020-06-01 22,314,271.22
Consumer Cyclical      2019-10-01    498,348.66
                       2019-11-01    619,041.02
                       2019-12-01    387,422.12
                       2020-01-01    433,164.34
                       2020-02-01    787,628.52
                       2020-03-01  4,772,653.53
                       2020-04-01  1,520,232.83
                       2020-05-01  1,324,027.47
                       2020-06-01  1,074,983.76
Consumer Defensive     2019-10-01 22,382,316.29
                       2019-11-01 15,562,290.42
                       2019-12-01 17,966,126.14
...                                         ...
Industrials            2020-04-01  8,620,570.76
                       2020-05-01  8,749,078.42
                       2020-06-01  8,323,476.06
Real Estate            2019-10-01  3,704,723.77
                       2019-11-01  2,295,100.80
                       2019-12-01  1,987,653.41
                       2020-01-01  4,367,405.83
                       2020-02-01  5,807,826.00
                       2020-03-01 16,012,671.73
                       2020-04-01  5,641,887.15
                       2020-05-01  3,676,540.57
                       2020-06-01  5,818,794.90
Technology             2019-10-01 15,904,522.18
                       2019-11-01  9,563,902.57
                       2019-12-01 16,448,808.31
                       2020-01-01 14,395,960.58
                       2020-02-01 29,878,186.59
                       2020-03-01 49,470,840.25
                       2020-04-01 34,965,628.27
                       2020-05-01 32,101,078.53
                       2020-06-01 25,993,384.14
Utilities              2019-10-01        769.93
                       2019-11-01        494.95
                       2019-12-01      1,869.83
                       2020-01-01      3,629.67
                       2020-02-01      3,299.70
                       2020-03-01      1,649.85
                       2020-04-01        989.91
                       2020-05-01        412.46
                       2020-06-01        962.41

[99 rows x 1 columns]

In [60]:
df4 = new_table.groupby(['sector(s)', 'time'])['difference',].sum().reset_index()

df4[df4['time'] == '2020-03-01']

,sector(s),time,difference
5,Basic Materials,2020-03-01,"24,549,660.96"
14,Communication Services,2020-03-01,"50,338,503.56"
23,Consumer Cyclical,2020-03-01,"4,772,653.53"
32,Consumer Defensive,2020-03-01,"71,572,707.41"
41,Energy,2020-03-01,"177,110,518.46"
50,Financial Services,2020-03-01,"42,789,558.88"
59,Healthcare,2020-03-01,"16,766,584.47"
68,Industrials,2020-03-01,"22,289,318.79"
77,Real Estate,2020-03-01,"16,012,671.73"
86,Technology,2020-03-01,"49,470,840.25"
